<a href="https://colab.research.google.com/github/maddogmikeb/Jira/blob/master/TimeInStatus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# install dependencies

!pip install -q atlassian-python-api

In [14]:
# Configure display

from google.colab import data_table
data_table.enable_dataframe_formatter()

In [15]:
# Log in

from IPython.core.display import display, HTML
from atlassian import Jira
from google.colab import userdata

jira = Jira(
  url=userdata.get('atlassian_host'),
  username=userdata.get('atlassian_username'),
  password=userdata.get('atlassian_apikey'),
  cloud=True
)

me = jira.myself()

display(HTML('<table><tr><td>' + me["displayName"] + '</td><td><img src="' + me["avatarUrls"]["32x32"] + '"/><td></tr></table>'))


Mike Burns


In [16]:
# Get all issues from jql
# Code to help debug -> https://github.com/atlassian-api/atlassian-python-api/blob/master/atlassian/jira.py

from IPython.display import clear_output, display
from atlassian import Jira

JQL = 'project = FDSEWMSR AND issuetype in (Bug, Story, Epic, Task) AND "Team[Team]" = d3706851-4fae-4b34-9a25-d4e10c5a45e4 and statuscategory = "done" ORDER BY Rank ASC'

limit = None

params = {}
if limit is not None:
  params["maxResults"] = int(limit)
params["fields"] = "key,created,resolutiondate,status,project"
params["jql"] = JQL
#params["expand"] = expand
url = jira.resource_url("search")
start = 0
results = []

while True:
  clear_output(wait=True)

  params["startAt"] = int(start)
  response = jira.get(url, params=params)
  if not response:
    break

  issues = response["issues"]
  results.extend(issues)
  total = int(response["total"])
  display("DBG: response: total={total} start={startAt} max={maxResults}".format(**response))
  # If we don't have a limit, and there's more to fetch, keep looping
  if limit is not None or total <= len(response["issues"]) + start:
    break
  start += len(issues)

clear_output()

In [18]:
# Get all the change logs and iterate through them to find all the status changes

from IPython.display import clear_output, display
from atlassian import Jira
import datetime
import pandas as pd
import numpy as np
import copy

issues = copy.deepcopy(results)

for issue in issues:
  clear_output(wait=True)
  display("DBG: checking key={key}".format(**issue))

  changelog = jira.get_issue_changelog(issue["key"])
  changes = []
  lastChange = issue["fields"]["created"]
  for log in changelog["values"]:
    for logitem in log["items"]:
      if logitem["field"].upper() == "STATUS":
        logitem["start"] = lastChange
        logitem["end"] = log["created"]
        lastChange = log["created"]
        changes += [
            {
              'statusid': logitem["from"],
              'status': logitem["fromString"],
              #'start': logitem["start"],
              #'end': logitem["end"],
              'total': (datetime.datetime.fromisoformat(logitem["end"]) - datetime.datetime.fromisoformat(logitem["start"])).total_seconds()
            }]
        #display(logitem)
  if len(changes) > 0:
    changes += [
      {
        'statusid': issue["fields"]["status"]["id"],
        'status': issue["fields"]["status"]["name"],
        #'start': lastChange,
        #'end': None,
        'total': float('inf')
      }]

    df = pd.DataFrame(changes)
    df.groupby('statusid', as_index=False)['total'].sum()
    df = df.reset_index()
    df = df.replace({None: np.nan})
    for index, row in df.iterrows():
      issue[row["statusid"] + "|" + row["status"]] = row["total"]

  # clean up fields
  issue["created"] = issue["fields"]["created"]
  issue["resolutiondate"] = issue["fields"]["resolutiondate"]
  issue["project"] = issue["fields"]["project"]["name"]
  issue["url"] = jira.url + "browse/" + issue["key"]
  del issue["fields"]
  del issue["expand"]
  del issue["self"]

clear_output()
#display( pd.DataFrame(changes) )

In [19]:
# Print

from IPython.core.display import display, HTML

import json
import pandas as pd
import numpy as np

#print(json.dumps(data, indent=2))

df = pd.DataFrame(issues)
df = df.reindex(sorted(df.columns, reverse=True), axis=1)
df = df.replace({None: np.nan})
# display(df)

df.to_excel("output.xlsx", index=False)

,url,resolutiondate,project,key,id,created,3|In Progress,10470|TDA Approval,10448|FDA Approval,10447|PMG Approval,...,10100|On Hold,10097|Release Ready,10094|Cancelled,10058|In analysis,10046|Completed,10019|Review,10007|Backlog,10006|Done,10005|In Review,10004|To Do
0,https://brisbanecitycouncil.atlassian.net/brow...,2025-01-08T17:24:20.122+1000,TRACE Program,FDSEWMSR-16553,82369,2024-12-09T13:21:04.428+1000,NaN,NaN,NaN,NaN,...,NaN,NaN,inf,NaN,NaN,NaN,2.606596e+06,NaN,NaN,NaN
1,https://brisbanecitycouncil.atlassian.net/brow...,2025-01-08T17:24:44.717+1000,TRACE Program,FDSEWMSR-16554,82378,2024-12-09T14:05:41.284+1000,NaN,NaN,NaN,NaN,...,NaN,NaN,inf,NaN,NaN,NaN,2.603943e+06,NaN,NaN,NaN
2,https://brisbanecitycouncil.atlassian.net/brow...,2025-01-08T17:24:59.202+1000,TRACE Program,FDSEWMSR-16555,82386,2024-12-09T15:19:08.426+1000,NaN,NaN,NaN,NaN,...,NaN,NaN,inf,NaN,NaN,NaN,2.599551e+06,NaN,NaN,NaN
3,https://brisbanecitycouncil.atlassian.net/brow...,2025-03-07T10:05:35.253+1000,TRACE Program,FDSEWMSR-13738,61155,2024-07-25T14:34:35.667+1000,NaN,NaN,NaN,NaN,...,8627291.729,NaN,inf,NaN,NaN,NaN,1.079657e+07,NaN,NaN,NaN
4,https://brisbanecitycouncil.atlassian.net/brow...,2024-11-29T10:12:50.955+1000,TRACE Program,FDSEWMSR-11728,46113,2024-04-16T12:40:05.362+1000,NaN,NaN,NaN,NaN,...,9411393.782,NaN,inf,NaN,NaN,NaN,1.019257e+07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,https://brisbanecitycouncil.atlassian.net/brow...,2024-12-13T08:33:03.685+1000,TRACE Program,FDSEWMSR-16518,81962,2024-12-05T08:18:14.796+1000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.327223e+03,NaN,3.510,NaN
94,https://brisbanecitycouncil.atlassian.net/brow...,2024-12-11T11:12:07.706+1000,TRACE Program,FDSEWMSR-16522,81980,2024-12-05T09:19:09.591+1000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.980595e+03,NaN,3893.507,NaN
95,https://brisbanecitycouncil.atlassian.net/brow...,2024-12-09T15:18:24.297+1000,TRACE Program,FDSEWMSR-16523,81998,2024-12-05T11:34:25.598+1000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.103630e+02,NaN,3.239,NaN
96,https://brisbanecitycouncil.atlassian.net/brow...,2025-01-15T14:11:44.095+1000,TRACE Program,FDSEWMSR-16629,83325,2024-12-16T15:19:32.403+1000,NaN,NaN,NaN,NaN,...,NaN,NaN,inf,NaN,NaN,NaN,1.493069e+05,NaN,NaN,NaN
